[![Binder](../img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignments/signal-exploration.ipynb)&emsp;
[![Script](../img/badge-script.svg)](/Teaching//assignments/signal-exploration.fsx)&emsp;
[![Notebook](../img/badge-notebook.svg)](/Teaching//assignments/signal-exploration.ipynb)

Group Name:

Student Name | Student Number
--- | ---
**1:Bernardo Manarte** |55810 &#32;
**2:Leonor Rodrigues Pereira** |48778 &#32;
**3:Marouan Kamoun** |53833 &#32;
**4:Rodrigo Simões** |53154 &#32;
**5:Vasco Calxito** |53960 &#32;



This is an assignment. You should work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.

Load libraries.



In [1]:
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET, 3.*"
#r "nuget: Plotly.NET.Interactive, 3.*"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

open System
open FSharp.Data
open FSharp.Stats
open Plotly.NET
open MathNet.Numerics.Statistics


Installed Packages FSharp.Data, 6.2.0 FSharp.Stats, 0.4.11 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `/Users/rodrigosimoes/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

### First, make sure that you're referencing the correct files.

You're going to write code to explore an investment signal. The investment signals can be downloaded from moodle. You are free to use whichever signal you want. It does not have to be the signal that you are suing for your final project.

Here I'm assuming that you have a class folder with this `signal-exploration.ipynb` notebook and a `data` folder inside of it. The folder hierarchy would look like below where you
have the below files and folders accessible:

```code
/class
    signal-exploration.ipynb
    id_and_return_data.csv
    zero_trades_252d.csv
    
```

First, make sure that our working directory is the source file directory.



In [2]:
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder


We assume the `id_and_return_data.csv` file and the signal csv file  are in the `data` folder. In this example the signal file is `zero_trades_252d.csv`. You should replace that file name with your signal file name.



In [3]:
let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "earnings_variability.csv" // "zero_trades_252d.csv"


## Input data

First, let's verify that the csv file exists where we think it will be.



In [4]:
if IO.File.Exists(ResolutionFolder + "/" + IdAndReturnsFilePath) &&
   IO.File.Exists(ResolutionFolder + "/" + MySignalFilePath) then 
    printfn "Success!!"
else
    let filesThere = IO.Directory.EnumerateFiles(
        ResolutionFolder,
        searchPattern = "*",
        searchOption=IO.SearchOption.AllDirectories)
    printfn "We did not find the files. Here are the files in your source directory.\n"
    filesThere |> Seq.iteri (printfn "%i. %A")


Success!!


Assuming that you got "Success!!" above this code below will work.

If my paths are correct, then this code should read the first few lines of the files.
If it doesn't show the first few lines, fix the above file paths.



In [5]:
IO.File.ReadLines(IdAndReturnsFilePath) |> Seq.truncate 5


[ id(string),eom(date),source(string),sizeGrp(string),obsMain(bool),exchMain(bool),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option), crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875, crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954, crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511210,36,0.285714,0.2813712985,3.9375,42.38325, crsp_85756,2000-01-31T00:00:00.0000000,CRSP,nano,1,1,true,066389,01,85756,15930,USA,USD,1,true,,11,,3,1,1.582,,6720,,48,0.190685,0.1863418699,10.25,16.2155 ]

In [6]:
IO.File.ReadLines(MySignalFilePath) |> Seq.truncate 5


[ id(string),eom(date),signal(float option), comp_001034_01,2008-12-31T00:00:00.0000000,1.5440442762, comp_001043_01,2000-01-31T00:00:00.0000000,0.5181897494, comp_001076_02,2010-12-31T00:00:00.0000000,0.1793233983, comp_001081_01,2007-10-31T00:00:00.0000000,1.1655154244 ]

Assuming the paths are defined correctly and you saw the first 5 rows above,
we can now read the data using the CSV provider that parses the fields in the file.

First define the Csv types from the sample files:



In [7]:
type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>


Now read in the data.



In [8]:
let idAndReturnsCsv = IdAndReturnsType.GetSample()

let mySignalCsv = MySignalType.GetSample()
    


Columns in the `idAndReturnsCsv` are:



In [9]:
idAndReturnsCsv.Headers


Value,"[ id(string), eom(date), source(string), sizeGrp(string), obsMain(bool), exchMain(bool), primarySec(bool), gvkey(string), iid(string), permno(int Option), permco(int Option), excntry(string), curcd(string), fx(string), common(bool), compTpci(string), crspShrcd(int Option), compExchg(string), crsp_exchcd(int Option), adjfct(float Option) ... (9 more) ]"


Columns in the `mySignalCsv` are:



In [10]:
mySignalCsv.Headers


Value,"[ id(string), eom(date), signal(float option) ]"


There are a lot of columns in the id and returns csv. You can look at the data documentation to figure out what they are.

Put the rows into a list (we're more familiar with lists).



In [11]:
let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList


### Distribution of unique stocks in the id and returns data

To get you started, I will walk you through some simple analysis of the id and returns data.

Count the total number of stocks.

First, look at a few ids



In [12]:
idAndReturnsRows
|> List.map (fun row -> row.Id)
|> List.truncate 5


[ crsp_86432, crsp_85640, crsp_86430, crsp_85756, crsp_50278 ]

Now count all of them.



In [13]:
idAndReturnsRows
|> List.map (fun row -> row.Id)
|> List.distinct
|> List.length


14000

Number of stocks each month.

First look at the date column



In [14]:
idAndReturnsRows
|> List.map (fun row -> row.Eom)
|> List.truncate 5


[ 2000-01-31 00:00:00Z, 2000-01-31 00:00:00Z, 2000-01-31 00:00:00Z, 2000-01-31 00:00:00Z, 2000-01-31 00:00:00Z ]

Group by month, then count per month.



In [15]:
let idAndReturnStocksPerMonth =
    let byMonth =
        idAndReturnsRows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in byMonth do
        let nStocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nStocks ]


Look at a first few months.



In [16]:
idAndReturnStocksPerMonth
|> List.sortBy (fun (month, nStocks) -> month) 
|> List.truncate 5


index value 0 (31/01/2000 00:00:00, 7076) Item1 2000-01-31 00:00:00Z Item2 7076 1 (29/02/2000 00:00:00, 7031) Item1 2000-02-29 00:00:00Z Item2 7031 2 (31/03/2000 00:00:00, 7021) Item1 2000-03-31 00:00:00Z Item2 7021 3 (30/04/2000 00:00:00, 7037) Item1 2000-04-30 00:00:00Z Item2 7037 4 (31/05/2000 00:00:00, 7053) Item1 2000-05-31 00:00:00Z Item2 7053

Look at the last few.



In [17]:
idAndReturnStocksPerMonth
|> List.sortByDescending (fun (month, nStocks) -> month)
|> List.truncate 5


index value 0 (31/12/2020 00:00:00, 4269) Item1 2020-12-31 00:00:00Z Item2 4269 1 (30/11/2020 00:00:00, 4216) Item1 2020-11-30 00:00:00Z Item2 4216 2 (31/10/2020 00:00:00, 4156) Item1 2020-10-31 00:00:00Z Item2 4156 3 (30/09/2020 00:00:00, 4129) Item1 2020-09-30 00:00:00Z Item2 4129 4 (31/08/2020 00:00:00, 4105) Item1 2020-08-31 00:00:00Z Item2 4105

Create a column chart showing the number of stocks per month (Plotly.net column chart [docs](https://plotly.net/02_1_bar-and-column-charts.html)).



In [18]:
idAndReturnStocksPerMonth
|> Chart.Column


<!-- Plotly chart will be drawn inside this DIV -->

Add some lables to the axes (Plotly.net axis styling [docs](https://plotly.net/01_0_axis-styling.html)).



In [19]:
idAndReturnStocksPerMonth
|> List.sortBy (fun (month, nStocks) -> month)
|> Chart.Column
|> Chart.withXAxisStyle (TitleText="Month")
|> Chart.withYAxisStyle (TitleText="Number of Stocks")


<!-- Plotly chart will be drawn inside this DIV -->

We have some different size groups already assigned in the data:



In [20]:
idAndReturnsRows
|> List.countBy (fun row -> row.SizeGrp)


index value 0 (micro, 486881) Item1 micro Item2 486881 1 (small, 263335) Item1 small Item2 263335 2 (nano, 197705) Item1 nano Item2 197705 3 (large, 173040) Item1 large Item2 173040 4 (mega, 97879) Item1 mega Item2 97879

Let's make a plot with separate bars for each group in 2015. You can read more about multiple charts in the Plotly.net [docs](https://plotly.net/01_2_multiple-charts.html).

We'll write a function. We need to give a type hint so that
it knows the type of the input data. If we didn't include the type hint, we'd get an error saying 'Lookup of indeterminate type ..' because it doesn't know the data type of the 'rows' input. The type hint the  `: list<IdAndReturnsType.Row>` part of the function definition.
This is saying we have a list of rows from the CsvProvider type that we defined earlier for this csv file data.



In [21]:
let countIdAndReturnsRows (rows: list<IdAndReturnsType.Row>) =
    let byMonth =
        rows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in byMonth do
        let nStocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nStocks ]


Look at the function output. It is a list of tuples where each tuple is a pair of month (`DateTime`) and the count (`int`).



In [22]:
idAndReturnsRows
|> countIdAndReturnsRows
|> List.truncate 3


index value 0 (31/01/2000 00:00:00, 7076) Item1 2000-01-31 00:00:00Z Item2 7076 1 (29/02/2000 00:00:00, 7031) Item1 2000-02-29 00:00:00Z Item2 7031 2 (31/03/2000 00:00:00, 7021) Item1 2000-03-31 00:00:00Z Item2 7021

Just for large caps.



In [23]:
let stockCountsLarge =
    let toPlot = 
        idAndReturnsRows
        |> List.filter (fun row -> 
            row.SizeGrp = "large" && 
            row.Eom.Year = 2015)
        |> countIdAndReturnsRows
    Chart.Column(toPlot, Name = "Large caps")

stockCountsLarge


<!-- Plotly chart will be drawn inside this DIV -->

Just for small caps.



In [24]:
let stockCountsSmall =
    let toPlot = 
        idAndReturnsRows
        |> List.filter (fun row -> 
            row.SizeGrp = "small" &&
            row.Eom.Year = 2015)
        |> countIdAndReturnsRows
    Chart.Column(toPlot, Name = "Small caps")

stockCountsSmall


<!-- Plotly chart will be drawn inside this DIV -->

combined:



In [25]:
[ stockCountsLarge; stockCountsSmall ]
|> Chart.combine


<!-- Plotly chart will be drawn inside this DIV -->

Now all groups



In [26]:
let stockCountsAllSizes =
    idAndReturnsRows
    |> List.filter (fun row -> row.Eom.Year = 2015)
    |> List.groupBy (fun row -> row.SizeGrp)
    |> List.map (fun (sizeGrp, rows) -> 
        let toPlot = countIdAndReturnsRows rows
        sizeGrp, toPlot)

// first few observations of all size Groups
stockCountsAllSizes
|> List.map (fun (sizeGroup, xs) ->
    sizeGroup, xs |> List.truncate 3)


index value 0 (micro, [(31/01/2015 00:00:00, 1583); (28/02/2015 00:00:00, 1532); (31/03/2015 00:00:00, 1509)]) Item1 micro Item2 index value 0 (31/01/2015 00:00:00, 1583) Item1 2015-01-31 00:00:00Z Item2 1583 1 (28/02/2015 00:00:00, 1532) Item1 2015-02-28 00:00:00Z Item2 1532 2 (31/03/2015 00:00:00, 1509) Item1 2015-03-31 00:00:00Z Item2 1509 1 (mega, [(31/01/2015 00:00:00, 410); (28/02/2015 00:00:00, 412); (31/03/2015 00:00:00, 410)]) Item1 mega Item2 index value 0 (31/01/2015 00:00:00, 410) Item1 2015-01-31 00:00:00Z Item2 410 1 (28/02/2015 00:00:00, 412) Item1 2015-02-28 00:00:00Z Item2 412 2 (31/03/2015 00:00:00, 410) Item1 2015-03-31 00:00:00Z Item2 410 2 (small, [(31/01/2015 00:00:00, 1050); (28/02/2015 00:00:00, 1058); (31/03/2015 00:00:00, 1031)]) Item1 small Item2 index value 0 (31/01/2015 00:00:00, 1050) Item1 2015-01-31 00:00:00Z Item2 1050 1 (28/02/2015 00:00:00, 1058) Item1 2015-02-28 00:00:00Z Item2 1058 2 (31/03/2015 00:00:00, 1031) Item1 2015-03-31 00:00:00Z Item2 1031 3 (large, [(31/01/2015 00:00:00, 706); (28/02/2015 00:00:00, 695); (31/03/2015 00:00:00, 700)]) Item1 large Item2 index value 0 (31/01/2015 00:00:00, 706) Item1 2015-01-31 00:00:00Z Item2 706 1 (28/02/2015 00:00:00, 695) Item1 2015-02-28 00:00:00Z Item2 695 2 (31/03/2015 00:00:00, 700) Item1 2015-03-31 00:00:00Z Item2 700 4 (nano, [(31/01/2015 00:00:00, 538); (28/02/2015 00:00:00, 580); (31/03/2015 00:00:00, 617)]) Item1 nano Item2 index value 0 (31/01/2015 00:00:00, 538) Item1 2015-01-31 00:00:00Z Item2 538 1 (28/02/2015 00:00:00, 580) Item1 2015-02-28 00:00:00Z Item2 580 2 (31/03/2015 00:00:00, 617) Item1 2015-03-31 00:00:00Z Item2 617

A combined chart.



In [27]:
stockCountsAllSizes
|> List.map (fun (sizeGrp, toPlot) -> 
    Chart.Column(toPlot, Name = sizeGrp))
|> Chart.combine

(** Same, but stacking each chart on top of eachother. *)   

stockCountsAllSizes
|> List.map (fun (sizeGrp, toPlot) -> 
    Chart.Column(toPlot, Name = sizeGrp))
|> Chart.SingleStack()


<!-- Plotly chart will be drawn inside this DIV -->

You should now have some a good idea of how to work with this data.

### Distribution of unique stocks in the your signal data

Do similar analysis as above, but for the your signal data.

> **Task:** Complete this function. It takes a list of `MySignalType.Row` as input and should return a list of the month and the integer count of unique stock ids that month (`list<DateTime * int>`).
> 



In [28]:
mySignalRows

index value 0 (comp_001034_01, 31/12/2008 00:00:00, Some(1.5440442762)) Item1 comp_001034_01 Item2 2008-12-31 00:00:00Z Item3 Some(1.5440442762) Value 1.5440442762 1 (comp_001043_01, 31/01/2000 00:00:00, Some(0.5181897494)) Item1 comp_001043_01 Item2 2000-01-31 00:00:00Z Item3 Some(0.5181897494) Value 0.5181897494 2 (comp_001076_02, 31/12/2010 00:00:00, Some(0.1793233983)) Item1 comp_001076_02 Item2 2010-12-31 00:00:00Z Item3 Some(0.1793233983) Value 0.1793233983 3 (comp_001081_01, 31/10/2007 00:00:00, Some(1.1655154244)) Item1 comp_001081_01 Item2 2007-10-31 00:00:00Z Item3 Some(1.1655154244) Value 1.1655154244 4 (comp_001095_01, 30/06/2002 00:00:00, Some(1.2286652155)) Item1 comp_001095_01 Item2 2002-06-30 00:00:00Z Item3 Some(1.2286652155) Value 1.2286652155 5 (comp_001111_02, 31/07/2008 00:00:00, Some(0.5204224947)) Item1 comp_001111_02 Item2 2008-07-31 00:00:00Z Item3 Some(0.5204224947) Value 0.5204224947 6 (comp_001117_02, 31/05/2003 00:00:00, Some(1.3434343589)) Item1 comp_001117_02 Item2 2003-05-31 00:00:00Z Item3 Some(1.3434343589) Value 1.3434343589 7 (comp_001117_02, 30/06/2003 00:00:00, Some(1.3434343589)) Item1 comp_001117_02 Item2 2003-06-30 00:00:00Z Item3 Some(1.3434343589) Value 1.3434343589 8 (comp_001117_02, 31/07/2003 00:00:00, Some(1.0415154697)) Item1 comp_001117_02 Item2 2003-07-31 00:00:00Z Item3 Some(1.0415154697) Value 1.0415154697 9 (comp_001117_02, 31/08/2003 00:00:00, Some(1.0415154697)) Item1 comp_001117_02 Item2 2003-08-31 00:00:00Z Item3 Some(1.0415154697) Value 1.0415154697 10 (comp_001117_02, 30/09/2003 00:00:00, Some(1.0415154697)) Item1 comp_001117_02 Item2 2003-09-30 00:00:00Z Item3 Some(1.0415154697) Value 1.0415154697 11 (comp_001117_02, 31/10/2003 00:00:00, Some(0.6296578334)) Item1 comp_001117_02 Item2 2003-10-31 00:00:00Z Item3 Some(0.6296578334) Value 0.6296578334 12 (comp_001117_02, 30/11/2003 00:00:00, Some(0.6296578334)) Item1 comp_001117_02 Item2 2003-11-30 00:00:00Z Item3 Some(0.6296578334) Value 0.6296578334 13 (comp_001117_02, 31/12/2003 00:00:00, Some(0.6296578334)) Item1 comp_001117_02 Item2 2003-12-31 00:00:00Z Item3 Some(0.6296578334) Value 0.6296578334 14 (comp_001117_02, 31/01/2004 00:00:00, Some(0.623650494)) Item1 comp_001117_02 Item2 2004-01-31 00:00:00Z Item3 Some(0.623650494) Value 0.623650494 15 (comp_001117_02, 29/02/2004 00:00:00, Some(0.623650494)) Item1 comp_001117_02 Item2 2004-02-29 00:00:00Z Item3 Some(0.623650494) Value 0.623650494 16 (comp_001117_02, 31/03/2004 00:00:00, Some(0.623650494)) Item1 comp_001117_02 Item2 2004-03-31 00:00:00Z Item3 Some(0.623650494) Value 0.623650494 17 (comp_001117_02, 30/04/2004 00:00:00, Some(1.6287993009)) Item1 comp_001117_02 Item2 2004-04-30 00:00:00Z Item3 Some(1.6287993009) Value 1.6287993009 18 (comp_001117_02, 31/05/2004 00:00:00, Some(1.6287993009)) Item1 comp_001117_02 Item2 2004-05-31 00:00:00Z Item3 Some(1.6287993009) Value 1.6287993009 19 (comp_001117_02, 30/06/2004 00:00:00, Some(1.6287993009)) Item1 comp_001117_02 Item2 2004-06-30 00:00:00Z Item3 Some(1.6287993009) Value 1.6287993009 ... (more)

In [29]:
// solution here
//let countMySignalRows (rows: list<MySignalType.Row>) =
    //failwith "I am not implemented yet"

let countMySignalRows (rows: list<MySignalType.Row>) =
    let byMonth =
        rows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in byMonth do
        let nStocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nStocks ]
    
mySignalRows
|> countMySignalRows
|> List.truncate 3



index value 0 (31/01/2000 00:00:00, 7076) Item1 2000-01-31 00:00:00Z Item2 7076 1 (29/02/2000 00:00:00, 7031) Item1 2000-02-29 00:00:00Z Item2 7031 2 (31/03/2000 00:00:00, 7021) Item1 2000-03-31 00:00:00Z Item2 7021

> **Task:** Create a column chart showing the number of stocks per month in your signal data csv file.
> 



In [30]:
// solution here
mySignalRows
|> countMySignalRows
|> Chart.Column
|> Chart.withXAxisStyle (TitleText="Month")
|> Chart.withYAxisStyle (TitleText="Number of Stocks")


<!-- Plotly chart will be drawn inside this DIV -->

You may have some stocks with missing data. If you have some stocks with missing signal data, the below code will return the first 3 observations.
If you do not have missing data it will return an empty list.



In [31]:
mySignalRows
|> List.choose (fun row -> 
    // Choose the rows where row.Signal is None.
    match row.Signal with
    | None -> Some row
    | Some signal -> None )
|> List.truncate 3


index value 0 (comp_002620_01, 30/11/2008 00:00:00, ) Item1 comp_002620_01 Item2 2008-11-30 00:00:00Z Item3 <null> 1 (comp_004424_01, 30/09/2000 00:00:00, ) Item1 comp_004424_01 Item2 2000-09-30 00:00:00Z Item3 <null> 2 (comp_004424_01, 31/10/2000 00:00:00, ) Item1 comp_004424_01 Item2 2000-10-31 00:00:00Z Item3 <null>

We can create a list that only contains stocks with non-missing signals. We define a record type to hold this data. The main change is making signal have `float` type instead of `Option<float>` because we're removing missing data.



In [32]:
type NonMissingSignal =
    {
        Id: string
        Eom: DateTime
        Signal: float
    }

let myNonMissingSignals =
    mySignalRows
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal -> 
            Some { Id = row.Id; Eom = row.Eom; Signal = signal })


> **Task:** Complete this function. It takes a list of `NonMissingSignal` records as input and should return a list of the month and the integer count of unique stock ids that month (`list<DateTime * int>`).
> 



In [33]:
// solution here
//let countMyNonMissingSignalRows (rows: list<NonMissingSignal>) =
    //failwith "I am not implemented yet"

let countMyNonMissingSignalRows (rows: List<NonMissingSignal>) =
    let byMonth =
        rows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in byMonth do
        let nStocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nStocks ]
    
myNonMissingSignals
|> countMyNonMissingSignalRows
|> List.truncate 3
    


index value 0 (31/01/2000 00:00:00, 4092) Item1 2000-01-31 00:00:00Z Item2 4092 1 (29/02/2000 00:00:00, 4056) Item1 2000-02-29 00:00:00Z Item2 4056 2 (31/03/2000 00:00:00, 4023) Item1 2000-03-31 00:00:00Z Item2 4023

> **Task:** Create a column chart showing the number of stocks per month in your signal data that **do not** have missing signals.
> 



In [34]:
// solution here
myNonMissingSignals
|> countMyNonMissingSignalRows
|> Chart.Column
|> Chart.withXAxisStyle (TitleText="Month")
|> Chart.withYAxisStyle (TitleText="Number of Stocks")

<!-- Plotly chart will be drawn inside this DIV -->

### Distribution of the signal

> **Task:** Compute the minimum, maximum, median, standard deviation, and average of the non-missing signals in your dataset.
> 



In [35]:
// solution here.
let myNonMissingSignalsList = myNonMissingSignals |> Seq.map (fun row -> row.Signal) |> List.ofSeq
let minimum = myNonMissingSignalsList |> Seq.min
let maximum = myNonMissingSignalsList |> Seq.max
let median = myNonMissingSignalsList |> Seq.median
let stdev = myNonMissingSignalsList |> Seq.stDev
let average = myNonMissingSignalsList |> Seq.average

// Print the results
printfn "Minimum: %f" minimum
printfn "Maximum: %f" maximum
printfn "Median: %f" median
printfn "Standard deviation: %f" stdev
printfn "Average: %f" average

Minimum: 0.003078
Maximum: 1715.593697
Median: 0.917013
Standard deviation: 5.643603
Average: 1.371681


It can also be useful to compute percentiles of the signal. You can calculate percentils using `MathNet.Numerics.Statistics` quantile function.



In [36]:
// 10th, 50th, and 90th percentiles
let pctlExamples = [0.1; 0.5; 0.9]

// you must have an array of values
let pctlExamplesData = 
    [ 10.0; -20.0; 0.1; -5.0; 7.0; 4.0]
    |> List.toArray 


Compute the percentiles.



In [37]:
let pctlExamplesComputed =    
    [ for pctl in pctlExamples do
        Statistics.quantileFunc pctlExamplesData pctl ]
pctlExamplesComputed


[ -20, 2.05, 10 ]

> **Task:** Compute the 1st, 10th, 50th, 90th, and 99th percentiles of the non-missing signals in your dataset. Once these percentiles are calculated them, assign the signals to the values below. Explain what you learn about the distribution. Is it uniformly distributed, a skewed distribution, are there outliers, etc.?
> 



In [ ]:
// solution here
let pctl = [0.01; 0.1; 0.5; 0.9; 0.99]

let pctlComputed =    
    [ for p in pctl do
        Statistics.quantileFunc myNonMissingSignalsList p ]

let signalP01: float = pctlComputed[0] //failwith "I am not implemented yet"
let signalP10: float = pctlComputed[1] //failwith "I am not implemented yet"
let signalP50: float = pctlComputed[2] //failwith "I am not implemented yet"
let signalP90: float = pctlComputed[3] //failwith "I am not implemented yet"
let signalP99: float = pctlComputed[4] //failwith "I am not implemented yet"

pctlComputed


[ 0.0687351745, 0.2820431179, 0.9170129994, 2.5681433883033344, 8.3537863998 ]

Based on the percentiles computed, we can observe that the distribution of the non-missing signals is right-skewed or positively skewed. This means that the majority of the data points lie to the left of the median and the tail of the distribution is longer on the right side.

Furthermore, we can see that there are significant differences in the signal values, with the 99th percentile being more than 12 times larger than the 1st percentile. This indicates that there might be some outliers in the dataset, which can have a significant impact on any analysis or modeling performed using this data.

> **Task:** Create a [histogram](https://plotly.net/04_0_histograms.html) showing the distribution of the signal for all stocks in your dataset that have non-missing signals. Limit the data to 2015 to make it easier to plot. Explain what you learn about the distribution. Is it uniformly distributed, are there outliers, etc. How do you see this in the plot, and is there anything new that you learned relative to the percentiles?
> 



In [ ]:
// solution here.
myNonMissingSignals
|> List.filter (fun row -> row.Eom.Year = 2015)
|> List.map (fun row -> row.Signal)
|> Chart.Histogram
|> Chart.withXAxisStyle (TitleText="Signal")
|> Chart.withYAxisStyle (TitleText="Count")


<!-- Plotly chart will be drawn inside this DIV -->

From the histogram, we can see that the distribution of the signal is heavily skewed to the right, with a few stocks having very high signals, while the majority have signals that are closer to zero. This is consistent with what we observed in the percentiles, where the median value of the signal was around 0.917, while the maximum signal value was over 1,700.

The histogram also shows that there are some outliers with very high signals. These outliers are likely stocks that are performing exceptionally well and could be good candidates for further analysis.

Compared to the percentiles, the histogram provides a clearer visual representation of the distribution of the signal values, highlighting the skewness of the data and the presence of outliers.

[Winsorizing](https://en.wikipedia.org/wiki/Winsorizing) is a technique to remove the influence of outliers from a dataset. Let's create a winsorized version of your data.

Assuming that you have defined the percentile above correctly, this will create a winsorized version of your signal dataset. It is winsorized at the 1st and 99th percentiles.



In [ ]:
let winsorizeSignals (signalOb: NonMissingSignal) =
    let newSignal =
        if signalOb.Signal < signalP01 then 
            signalP01
        elif signalOb.Signal > signalP99 then
            signalP99
        else
            signalOb.Signal
    // copy and update the observation with the
    // winsorized signal.
    { signalOb with Signal = newSignal }


Test on a random signal



In [ ]:
winsorizeSignals myNonMissingSignals[99]


Id,comp_001478_01
Eom,2009-10-31 00:00:00Z
Signal,0.9286934995


do for all



In [ ]:
let myWinsorizedSignals =
    myNonMissingSignals
    |> List.map winsorizeSignals


> **Task:** Create a [histogram](https://plotly.net/04_0_histograms.html) showing the distribution of the **winsorized signals** for all stocks in your dataset. Limit the data to 2015 to make it easier to plot. Explain what you learn about the distribution. Is it uniformly distributed, are there outliers, etc. How do you see this in the plot, and is there anything new that you learned relative to the percentiles and non-winsorized histogram?
> 



In [ ]:
// solution here.
myWinsorizedSignals
|> List.filter (fun row -> row.Eom.Year = 2015)
|> List.map (fun row -> row.Signal)
|> Chart.Histogram
|> Chart.withXAxisStyle (TitleText="Signal")
|> Chart.withYAxisStyle (TitleText="Count")

<!-- Plotly chart will be drawn inside this DIV -->

From the winsorized signals histogram, we can observe that the distribution is now more symmetric compared to the original signal distribution. This is because we have limited the extreme values to the 1st and 99th percentiles, which helps to reduce the influence of the outliers.

We can also see that the range of the signal values has been significantly reduced. This is reflected in the smaller width of the histogram compared to the non-winsorized signal histogram.

Compared to the percentiles computed earlier, we see that the winsorized signals' distribution is much closer to a normal distribution with no significant outliers. This is because the winsorization process helps to remove the extreme values that can skew the distribution.

Overall, we can conclude that winsorizing the signals has helped to normalize the distribution and reduce the influence of the outliers. This can be useful in statistical analysis, where we want to remove the influence of outliers and get a better estimate of the central tendency of the data.

> **Task:** Create a map collection called `byStockMonthIdAndReturnMap` where the key is a tuple of stock id as string and month as DateTime (`string * DateTime`) and the value is an `IdAndReturnsType.Row`.
> 

**Note:** I have added a type constraint of `: Map<(string * DateTime), IdAndReturnsType.Row>` to make sure that the type of the map is correct. If you fill in code below, you will get a type mismatch error until your code is correct. You don't generally need these type constraints, but I am putting it here to make the compiler check that you produce the output that I am asking for.

**Hint:** we did things like this in the momentum signal lecture. There's also a practice quiz on map collections.



In [ ]:
// solution here

let byStockMonthIdAndReturnMap: Map<string * DateTime, IdAndReturnsType.Row> =
   idAndReturnsRows
    |> List.map (fun row -> ((row.Id,DateTime(row.Eom.Year, row.Eom.Month, 1)), row))
    |> Map
    //failwith "you haven't created your map collection." 
byStockMonthIdAndReturnMap

key value (comp_001034_01, 01/12/2008 00:00:00) Item1 comp_001034_01 Item2 2008-12-01 00:00:00Z (comp_001034_01, 31/12/2008 00:00:00, COMPUSTAT, large, True, True, True, 001034, 01, , , USA, USD, 1, True, 0, , 11, , Some(1), Some(41.882), Some(35202010), , , , Some(0.024931), Some(0.0249017502), Some(36.94), Some(1547.12108)) Item1 comp_001034_01 Item2 2008-12-31 00:00:00Z Item3 COMPUSTAT Item4 large Item5 True Item6 True Item7 True Rest (001034, 01, , , USA, USD, 1, True, 0, , 11, , Some(1), Some(41.882), Some(35202010), , , , Some(0.024931), Some(0.0249017502), Some(36.94), Some(1547.12108)) Item1 001034 Item2 01 Item3 <null> Item4 <null> Item5 USA Item6 USD Item7 1 Rest (True, 0, , 11, , Some(1), Some(41.882), Some(35202010), , , , Some(0.024931), Some(0.0249017502), Some(36.94), Some(1547.12108)) Item1 True Item2 0 Item3 <null> Item4 11 Item5 <null> Item6 Some(1) Value 1 Item7 Some(41.882) Value 41.882 Rest (Some(35202010), , , , Some(0.024931), Some(0.0249017502), Some(36.94), Some(1547.12108)) Item1 Some(35202010) Value 35202010 Item2 <null> Item3 <null> Item4 <null> Item5 Some(0.024931) Value 0,024931 Item6 Some(0.0249017502) Value 0,0249017502 Item7 Some(36.94) Value 36,94 Rest (Some(1547.12108)) Item1 Some(1547.12108) (comp_001043_01, 01/01/2000 00:00:00) Item1 comp_001043_01 Item2 2000-01-01 00:00:00Z (comp_001043_01, 31/01/2000 00:00:00, COMPUSTAT, nano, True, True, True, 001043, 01, , , USA, USD, 1, True, 0, , 12, , Some(1), Some(2.848), , , , , Some(0.111111), Some(0.1067681111), Some(1.25), Some(3.56)) Item1 comp_001043_01 Item2 2000-01-31 00:00:00Z Item3 COMPUSTAT Item4 nano Item5 True Item6 True Item7 True Rest (001043, 01, , , USA, USD, 1, True, 0, , 12, , Some(1), Some(2.848), , , , , Some(0.111111), Some(0.1067681111), Some(1.25), Some(3.56)) Item1 001043 Item2 01 Item3 <null> Item4 <null> Item5 USA Item6 USD Item7 1 Rest (True, 0, , 12, , Some(1), Some(2.848), , , , , Some(0.111111), Some(0.1067681111), Some(1.25), Some(3.56)) Item1 True Item2 0 Item3 <null> Item4 12 Item5 <null> Item6 Some(1) Value 1 Item7 Some(2.848) Value 2.848 Rest (, , , , Some(0.111111), Some(0.1067681111), Some(1.25), Some(3.56)) Item1 <null> Item2 <null> Item3 <null> Item4 <null> Item5 Some(0.111111) Value 0,111111 Item6 Some(0.1067681111) Value 0,1067681111 Item7 Some(1.25) Value 1,25 Rest (Some(3.56)) Item1 Some(3.56) (comp_001076_02, 01/12/2010 00:00:00) Item1 comp_001076_02 Item2 2010-12-01 00:00:00Z (comp_001076_02, 31/12/2010 00:00:00, COMPUSTAT, small, True, True, True, 001076, 02, , , USA, USD, 1, True, 0, , 11, , Some(1), Some(69.428), Some(25504060), Some(7359), Some(532299), Some(34), Some(0.034569), Some(0.0344291383), Some(20.65), Some(1433.6882)) Item1 comp_001076_02 Item2 2010-12-31 00:00:00Z Item3 COMPUSTAT Item4 small Item5 True Item6 True Item7 True Rest (001076, 02, , , USA, USD, 1, True, 0, , 11, , Some(1), Some(69.428), Some(25504060), Some(7359), Some(532299), Some(34), Some(0.034569), Some(0.0344291383), Some(20.65), Some(1433.6882)) Item1 001076 Item2 02 Item3 <null> Item4 <null> Item5 USA Item6 USD Item7 1 Rest (True, 0, , 11, , Some(1), Some(69.428), Some(25504060), Some(7359), Some(532299), Some(34), Some(0.034569), Some(0.0344291383), Some(20.65), Some(1433.6882)) Item1 True Item2 0 Item3 <null> Item4 11 Item5 <null> Item6 Some(1) Value 1 Item7 Some(69.428) Value 69.428 Rest (Some(25504060), Some(7359), Some(532299), Some(34), Some(0.034569), Some(0.0344291383), Some(20.65), Some(1433.6882)) Item1 Some(25504060) Value 25504060 Item2 Some(7359) Value 7359 Item3 Some(532299) Value 532299 Item4 Some(34) Value 34 Item5 Some(0.034569) Value 0,034569 Item6 Some(0.0344291383) Value 0,0344291383 Item7 Some(20.65) Value 20,65 Rest (Some(1433.6882)) Item1 Some(1433.6882) (comp_001081_01, 01/10/2007 00:00:00) Item1 comp_001081_01 Item2 2007-10-01 00:00:00Z (comp_001081_01, 31/10/2007 00:00:00, COMPUSTAT, small, True, True, True, 001081, 01, , , USA, USD, 1, True, 0, , 11, , Some(1), Some(440.175), , , , , Some(0.337

> **Task:** Create a [histogram](https://plotly.net/04_0_histograms.html) showing the distribution of the **winsorized signals** for only **small-cap stocks** in your dataset. Limit the data to 2015 to make it easier to plot.
> 

**Hint:** if you have a stock and it's signal in a particular month, the `byStockMonthIdAndReturnMap` is useful for looking up thinks about the stock that month.)



In [ ]:
// Filter the data for small-cap stocks in 2015
let smallCap2015Rows =
    idAndReturnsRows
    |> List.filter(fun row -> row.SizeGrp = "small" && row.Eom.Year = 2015)

// Collect the winsorized signals for small-cap stocks in 2015
let smallCapWinsorizedSignals2015 = 
    smallCap2015Rows
    |> List.choose(fun row -> 
        let key = (row.Id, DateTime(row.Eom.Year, row.Eom.Month, 1))
        match Map.tryFind key byStockMonthIdAndReturnMap with
        | Some m -> 
            let signal = winsorizeSignals {Id = m.Id; Eom = m.Eom; Signal = m.Ret.Value} // Adjust this part to match your winsorizeSignals function implementation
            Some signal
        | None -> None)

// Create a histogram of winsorized signals for small-cap stocks in 2015
let smallCapHistogram =
    smallCapWinsorizedSignals2015
    |> List.filter(fun row -> row.Id <> "")
    |> List.map(fun row -> row.Signal)
    |> Chart.Histogram
    |> Chart.withXAxisStyle("Winsorized Signal")
    |> Chart.withYAxisStyle("Count")

smallCapHistogram


<!-- Plotly chart will be drawn inside this DIV -->

> **Task:** Create a [histogram](https://plotly.net/04_0_histograms.html) showing the distribution of the **winsorized signals** for only **large-cap stocks** in your dataset. Limit the data to 2015 to make it easier to plot.
> 



In [ ]:
// Filter the data for small-cap stocks in 2015
let largeCap2015Rows =
    idAndReturnsRows
    |> List.filter(fun row -> row.SizeGrp = "large" && row.Eom.Year = 2015)

// Collect the winsorized signals for small-cap stocks in 2015
let largeCapWinsorizedSignals2015 = 
    largeCap2015Rows
    |> List.choose(fun row -> 
        let key = (row.Id, DateTime(row.Eom.Year, row.Eom.Month, 1))
        match Map.tryFind key byStockMonthIdAndReturnMap with
        | Some m -> 
            let signal = winsorizeSignals {Id = m.Id; Eom = m.Eom; Signal = m.Ret.Value} // Adjust this part to match your winsorizeSignals function implementation
            Some signal
        | None -> None)

// Create a histogram of winsorized signals for small-cap stocks in 2015
let largeCapHistogram =
    largeCapWinsorizedSignals2015
    |> List.filter(fun row -> row.Id <> "")
    |> List.map(fun row -> row.Signal)
    |> Chart.Histogram
    |> Chart.withXAxisStyle("Winsorized Signal")
    |> Chart.withYAxisStyle("Count")

largeCapHistogram


<!-- Plotly chart will be drawn inside this DIV -->

> **Task:** Compare and contrast the histograms for the **small-cap** and **large-cap** stocks. Are there any differences? If we wanted to sort stocks based on the signal, do you think that we would end up with stocks that have different average sizes in the low and high signal portfolios?
> 


Based on the analysis, it appears that small-cap stocks have a higher frequency around 0, and their Winsorized Signal is much lower compared to large-cap stocks. While small-cap stocks tend to be more concentrated near lower frequencies, there are fewer of them overall when compared to large-cap stocks.

If we were to sort the stocks by descending signals, we would find more small-cap stocks at the beginning of the list, with larger-cap stocks dominating as we move towards a 0.2 Winsorized Signal. Ultimately, only large-cap stocks would be present beyond this point.



### Towards portfolios.

> **Task:** Using your winsorized list of signals, group your stocks by month. Assign this result to a value named `byStockMonthSignals` that is a list of `DateTime * list<NonMissingSignal>` tuples. The first thing in the tuple is the month and the second thing is a list of `NonMissingSignal` records for all stocks in that month.
> 



In [ ]:
let byStockMonthSignals: list<DateTime * list<NonMissingSignal>> =
    myNonMissingSignals
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1 ))
byStockMonthSignals

index value 0 (01/12/2008 00:00:00, [{ Id = "comp_001034_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.544044276 }; { Id = "comp_007799_01"\n Eom = 31/12/2008 00:00:00\n Signal = 2.830487483 }; { Id = "comp_009551_02"\n Eom = 31/12/2008 00:00:00\n Signal = 1.322337199 }; ... ]) Item1 2008-12-01 00:00:00Z Item2 index value 0 { Id = "comp_001034_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.544044276 } Id comp_001034_01 Eom 2008-12-31 00:00:00Z Signal 1.5440442762 1 { Id = "comp_007799_01"\n Eom = 31/12/2008 00:00:00\n Signal = 2.830487483 } Id comp_007799_01 Eom 2008-12-31 00:00:00Z Signal 2.8304874826 2 { Id = "comp_009551_02"\n Eom = 31/12/2008 00:00:00\n Signal = 1.322337199 } Id comp_009551_02 Eom 2008-12-31 00:00:00Z Signal 1.3223371986 3 { Id = "comp_010364_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.8147440355 } Id comp_010364_01 Eom 2008-12-31 00:00:00Z Signal 0.8147440355 4 { Id = "comp_011286_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.326702511 } Id comp_011286_01 Eom 2008-12-31 00:00:00Z Signal 1.3267025111 5 { Id = "comp_011450_01"\n Eom = 31/12/2008 00:00:00\n Signal = 9.072655173 } Id comp_011450_01 Eom 2008-12-31 00:00:00Z Signal 9.0726551728 6 { Id = "comp_012717_01"\n Eom = 31/12/2008 00:00:00\n Signal = 2.986611497 } Id comp_012717_01 Eom 2008-12-31 00:00:00Z Signal 2.9866114969 7 { Id = "comp_012785_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.5987160206 } Id comp_012785_01 Eom 2008-12-31 00:00:00Z Signal 0.5987160206 8 { Id = "comp_012884_01"\n Eom = 31/12/2008 00:00:00\n Signal = 2.65175069 } Id comp_012884_01 Eom 2008-12-31 00:00:00Z Signal 2.65175069 9 { Id = "comp_013409_01"\n Eom = 31/12/2008 00:00:00\n Signal = 2.153909401 } Id comp_013409_01 Eom 2008-12-31 00:00:00Z Signal 2.1539094011 10 { Id = "comp_013520_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.506993748 } Id comp_013520_01 Eom 2008-12-31 00:00:00Z Signal 1.5069937478 11 { Id = "comp_017173_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.4927224445 } Id comp_017173_01 Eom 2008-12-31 00:00:00Z Signal 0.4927224445 12 { Id = "comp_017388_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.1956437 } Id comp_017388_01 Eom 2008-12-31 00:00:00Z Signal 1.1956436997 13 { Id = "comp_018361_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.7920822487 } Id comp_018361_01 Eom 2008-12-31 00:00:00Z Signal 0.7920822487 14 { Id = "comp_018385_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.1270902829 } Id comp_018385_01 Eom 2008-12-31 00:00:00Z Signal 0.1270902829 15 { Id = "comp_019129_02"\n Eom = 31/12/2008 00:00:00\n Signal = 0.4071861012 } Id comp_019129_02 Eom 2008-12-31 00:00:00Z Signal 0.4071861012 16 { Id = "comp_019658_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.7454101867 } Id comp_019658_01 Eom 2008-12-31 00:00:00Z Signal 0.7454101867 17 { Id = "comp_020134_01"\n Eom = 31/12/2008 00:00:00\n Signal = 0.7519716024 } Id comp_020134_01 Eom 2008-12-31 00:00:00Z Signal 0.7519716024 18 { Id = "comp_022140_02"\n Eom = 31/12/2008 00:00:00\n Signal = 0.7760258731 } Id comp_022140_02 Eom 2008-12-31 00:00:00Z Signal 0.7760258731 19 { Id = "comp_023170_01"\n Eom = 31/12/2008 00:00:00\n Signal = 1.203820029 } Id comp_023170_01 Eom 2008-12-31 00:00:00Z Signal 1.2038200289 ... (more) 1 (01/01/2000 00:00:00, [{ Id = "comp_001043_01"\n Eom = 31/01/2000 00:00:00\n Signal = 0.5181897494 }; { Id = "comp_001308_02"\n Eom = 31/01/2000 00:00:00\n Signal = 98.19897013 }; { Id = "comp_004045_01"\n Eom = 31/01/2000 00:00:00\n Signal = 0.8753499631 }; ... ]) Item1 2000-01-01 00:00:00Z Item2 index value 0 { Id = "comp_001043_01"\n Eom = 31/01/2000 00:00:00\n Signal = 0.5181897494 } Id comp_001043_01 Eom 2000-01-31 00:00:00Z Signal 0.5181897494 1 { Id = "comp_001308_02"\n Eom = 31/01/2000 00:00:00\n Signal = 98.19897013 } Id comp_001308_02 Eom 2000-01-31 00:00:00Z Signal 98.19897013 2 { Id = "comp_004045_01"\n Eom = 31/01/2000 00:00:00\n Signal = 0.8753499631 } Id comp_004045_01 Eom 2000-01-31 00:00:00Z Signal 0.8753499631 3 { Id = "comp_004485_01"\n Eom = 31/01/2000 00:00:00\n Signal = 0.34779

Based on the histograms of winsorized signals for small-cap and large-cap stocks, there are some noticeable differences in their distributions. Specifically, for small-cap stocks, the frequency of signals around 0 is much higher than for large-cap stocks. Additionally, for signals higher/lower than 0.5/-0.5, it appears that the frequency for large-cap stocks is higher.

If we were to sort stocks based on the signal, it is possible that we would end up with portfolios that have different average sizes in the low and high signal portfolios. The reason for this is that small-cap stocks may tend to have higher (or lower) signal values than large-cap stocks, which could lead to differences in portfolio composition if we use the signal to sort stocks. However, in order to fully understand how the signal would affect portfolio composition, we would need to analyze the correlation between the signal and the size of the stocks and perform portfolio simulations to see how the portfolio characteristics change when the signal is used for sorting stocks.

Now assuming `byStockMonthSignals` is correct, we'll sort the stocks each month from smallest to largest based on the signal that month. Then split the stocks into 3 equal-sized portfolios (aka terciles) based on the sorted signal. We'll create a `SortedPort` record for each portfolio and assign the list to a value named `terciles`.



In [ ]:
type SortedPort =
    { Portfolio: int
      Eom: DateTime
      Stocks: list<NonMissingSignal> }

let terciles =
    byStockMonthSignals
    |> List.collect (fun (eom, signals) ->
        let sortedSignals =
            signals
            |> List.sortBy (fun signalOb -> signalOb.Signal)
            |> List.splitInto 3
        sortedSignals
        |> List.mapi (fun i p -> 
            { Portfolio = i + 1
              Eom = eom
              Stocks = p }))


look at the first portfolio



In [ ]:
terciles[0]


{ Portfolio = 1\n Eom = 01/12/2008 00:00:00\n Stocks =\n [{ Id = "crsp_86250"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0138972399 }; { Id = "crsp_80007"\n Eom = 31/12/2008 00:00:00\n Signal = 0.018108817 };\n { Id = "cr... Portfolio 1 Eom 2008-12-01 00:00:00Z Stocks index value 0 { Id = "crsp_86250"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0138972399 } Id crsp_86250 Eom 2008-12-31 00:00:00Z Signal 0.0138972399 1 { Id = "crsp_80007"\n Eom = 31/12/2008 00:00:00\n Signal = 0.018108817 } Id crsp_80007 Eom 2008-12-31 00:00:00Z Signal 0.018108817 2 { Id = "crsp_77898"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0213731118 } Id crsp_77898 Eom 2008-12-31 00:00:00Z Signal 0.0213731118 3 { Id = "crsp_82654"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0228563506 } Id crsp_82654 Eom 2008-12-31 00:00:00Z Signal 0.0228563506 4 { Id = "crsp_84108"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0234484665 } Id crsp_84108 Eom 2008-12-31 00:00:00Z Signal 0.0234484665 5 { Id = "crsp_84058"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0260056721 } Id crsp_84058 Eom 2008-12-31 00:00:00Z Signal 0.0260056721 6 { Id = "crsp_90274"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0282247051 } Id crsp_90274 Eom 2008-12-31 00:00:00Z Signal 0.0282247051 7 { Id = "crsp_89307"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0290716092 } Id crsp_89307 Eom 2008-12-31 00:00:00Z Signal 0.0290716092 8 { Id = "crsp_11593"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0299417548 } Id crsp_11593 Eom 2008-12-31 00:00:00Z Signal 0.0299417548 9 { Id = "crsp_89615"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0306467565 } Id crsp_89615 Eom 2008-12-31 00:00:00Z Signal 0.0306467565 10 { Id = "crsp_52792"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0337344325 } Id crsp_52792 Eom 2008-12-31 00:00:00Z Signal 0.0337344325 11 { Id = "crsp_89428"\n Eom = 31/12/2008 00:00:00\n Signal = 0.034791273 } Id crsp_89428 Eom 2008-12-31 00:00:00Z Signal 0.034791273 12 { Id = "crsp_72996"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0408430113 } Id crsp_72996 Eom 2008-12-31 00:00:00Z Signal 0.0408430113 13 { Id = "crsp_87146"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0408811979 } Id crsp_87146 Eom 2008-12-31 00:00:00Z Signal 0.0408811979 14 { Id = "crsp_38033"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0411290831 } Id crsp_38033 Eom 2008-12-31 00:00:00Z Signal 0.0411290831 15 { Id = "crsp_85627"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0456309662 } Id crsp_85627 Eom 2008-12-31 00:00:00Z Signal 0.0456309662 16 { Id = "crsp_38703"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0486409369 } Id crsp_38703 Eom 2008-12-31 00:00:00Z Signal 0.0486409369 17 { Id = "crsp_91575"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0496296455 } Id crsp_91575 Eom 2008-12-31 00:00:00Z Signal 0.0496296455 18 { Id = "crsp_87168"\n Eom = 31/12/2008 00:00:00\n Signal = 0.0499439923 } Id crsp_87168 Eom 2008-12-31 00:00:00Z Signal 0.0499439923 19 { Id = "crsp_80072"\n Eom = 31/12/2008 00:00:00\n Signal = 0.05030805 } Id crsp_80072 Eom 2008-12-31 00:00:00Z Signal 0.05030805 ... (more)

look at the last portfolio



In [ ]:
terciles |> List.last


{ Portfolio = 3\n Eom = 01/02/2009 00:00:00\n Stocks =\n [{ Id = "crsp_11533"\n Eom = 28/02/2009 00:00:00\n Signal = 1.135671156 }; { Id = "crsp_20598"\n Eom = 28/02/2009 00:00:00\n Signal = 1.13568362 };\n { Id = "crsp_9... Portfolio 3 Eom 2009-02-01 00:00:00Z Stocks index value 0 { Id = "crsp_11533"\n Eom = 28/02/2009 00:00:00\n Signal = 1.135671156 } Id crsp_11533 Eom 2009-02-28 00:00:00Z Signal 1.135671156 1 { Id = "crsp_20598"\n Eom = 28/02/2009 00:00:00\n Signal = 1.13568362 } Id crsp_20598 Eom 2009-02-28 00:00:00Z Signal 1.13568362 2 { Id = "crsp_90333"\n Eom = 28/02/2009 00:00:00\n Signal = 1.135754448 } Id crsp_90333 Eom 2009-02-28 00:00:00Z Signal 1.1357544485 3 { Id = "crsp_82311"\n Eom = 28/02/2009 00:00:00\n Signal = 1.135805245 } Id crsp_82311 Eom 2009-02-28 00:00:00Z Signal 1.1358052448 4 { Id = "crsp_30648"\n Eom = 28/02/2009 00:00:00\n Signal = 1.136199194 } Id crsp_30648 Eom 2009-02-28 00:00:00Z Signal 1.1361991943 5 { Id = "crsp_85259"\n Eom = 28/02/2009 00:00:00\n Signal = 1.136376289 } Id crsp_85259 Eom 2009-02-28 00:00:00Z Signal 1.1363762886 6 { Id = "crsp_80446"\n Eom = 28/02/2009 00:00:00\n Signal = 1.136493035 } Id crsp_80446 Eom 2009-02-28 00:00:00Z Signal 1.1364930346 7 { Id = "crsp_90539"\n Eom = 28/02/2009 00:00:00\n Signal = 1.13716336 } Id crsp_90539 Eom 2009-02-28 00:00:00Z Signal 1.1371633603 8 { Id = "crsp_89413"\n Eom = 28/02/2009 00:00:00\n Signal = 1.138140642 } Id crsp_89413 Eom 2009-02-28 00:00:00Z Signal 1.1381406424 9 { Id = "crsp_44601"\n Eom = 28/02/2009 00:00:00\n Signal = 1.13961969 } Id crsp_44601 Eom 2009-02-28 00:00:00Z Signal 1.13961969 10 { Id = "crsp_78527"\n Eom = 28/02/2009 00:00:00\n Signal = 1.140298239 } Id crsp_78527 Eom 2009-02-28 00:00:00Z Signal 1.1402982387 11 { Id = "crsp_78741"\n Eom = 28/02/2009 00:00:00\n Signal = 1.140471422 } Id crsp_78741 Eom 2009-02-28 00:00:00Z Signal 1.1404714218 12 { Id = "crsp_91400"\n Eom = 28/02/2009 00:00:00\n Signal = 1.140785079 } Id crsp_91400 Eom 2009-02-28 00:00:00Z Signal 1.140785079 13 { Id = "crsp_45495"\n Eom = 28/02/2009 00:00:00\n Signal = 1.141210641 } Id crsp_45495 Eom 2009-02-28 00:00:00Z Signal 1.1412106407 14 { Id = "crsp_88448"\n Eom = 28/02/2009 00:00:00\n Signal = 1.141708708 } Id crsp_88448 Eom 2009-02-28 00:00:00Z Signal 1.141708708 15 { Id = "crsp_76712"\n Eom = 28/02/2009 00:00:00\n Signal = 1.142167059 } Id crsp_76712 Eom 2009-02-28 00:00:00Z Signal 1.1421670591 16 { Id = "crsp_88409"\n Eom = 28/02/2009 00:00:00\n Signal = 1.142175539 } Id crsp_88409 Eom 2009-02-28 00:00:00Z Signal 1.142175539 17 { Id = "crsp_10693"\n Eom = 28/02/2009 00:00:00\n Signal = 1.142383238 } Id crsp_10693 Eom 2009-02-28 00:00:00Z Signal 1.1423832385 18 { Id = "crsp_63546"\n Eom = 28/02/2009 00:00:00\n Signal = 1.142866943 } Id crsp_63546 Eom 2009-02-28 00:00:00Z Signal 1.1428669429 19 { Id = "crsp_33188"\n Eom = 28/02/2009 00:00:00\n Signal = 1.144646082 } Id crsp_33188 Eom 2009-02-28 00:00:00Z Signal 1.1446460816 ... (more)

> **Task:** Using `terciles`, compute the average signal in each tercile portfolio each month. Plot a combined (`Chart.combine`) line chart (`Chart.line`) showing the average signal for each tercile portfolio from the start to the end of the sample. What do you learn? Is the average signal in each tercile constant throughout the sample, or does it vary over time?
> 


In [ ]:
let generatePortfolioChart (portId: int) =
    terciles
    |> List.filter (fun tercile -> tercile.Portfolio = portId)
    |> List.map (fun data ->
        let meanSignal = data.Stocks |> List.averageBy (fun s -> s.Signal)
        data.Eom, meanSignal)
    |> List.sortBy (fun element -> fst element)
    |> Chart.Line
    |> Chart.withTraceInfo($"Portfolio {portId}")

let combinedPortfolioCharts =
    [ for portId in 1 .. 3 -> generatePortfolioChart portId ]
    |> List.toArray

Chart.combine combinedPortfolioCharts
|> Chart.withXAxisStyle("Date")
|> Chart.withYAxisStyle("Average Signal")

<!-- Plotly chart will be drawn inside this DIV -->


> **Task:** Using `byStockMonthSignals`, sort the stocks each month from smallest to largest based on the signal that month. Then split the stocks into 5 equal-sized portfolios (aka quintiles) based on the sorted signal. Create a `SortedPort` record for each portfolio and assign the list to a value named `quintiles`.
> 



In [ ]:
// solution here
let quintiles: list<SortedPort> =
    byStockMonthSignals
    |> List.collect (fun (month, monthSignals) ->
        let orderedSignals =
            monthSignals
            |> List.sortBy (fun signalData -> signalData.Signal)
            |> List.chunkBySize 5
        orderedSignals
        |> List.mapi (fun index portfolio -> 
            { Portfolio = index + 1
              Eom = month
              Stocks = portfolio }))



> **Task:** Filter `quintiles` to the quintile portfolio of stocks each month that has the lowest signal value. This should be stocks where `SortedPort.Portfolio = 1`. Assign the filtered list to a value named `bottomQuintile`.
> 



In [ ]:
let filterLowestSignalQuintile (quintileData: list<SortedPort>): list<SortedPort> =
    quintileData
    |> List.filter (fun entry -> entry.Portfolio = 1)
    |> List.map (fun entry ->
        let minSignalStock = entry.Stocks |> List.minBy (fun stock -> stock.Signal)
        { Portfolio = 1
          Eom = entry.Eom
          Stocks = [minSignalStock] } )
    |> List.sortBy (fun entry -> entry.Eom)

let bottomQuintile = filterLowestSignalQuintile quintiles
bottomQuintile


index value 0 { Portfolio = 1\n Eom = 01/01/2000 00:00:00\n Stocks = [{ Id = "crsp_87146"\n Eom = 31/01/2000 00:00:00\n Signal = 0.0095226144 }] } Portfolio 1 Eom 2000-01-01 00:00:00Z Stocks index value 0 { Id = "crsp_87146"\n Eom = 31/01/2000 00:00:00\n Signal = 0.0095226144 } Id crsp_87146 Eom 2000-01-31 00:00:00Z Signal 0.0095226144 1 { Portfolio = 1\n Eom = 01/02/2000 00:00:00\n Stocks = [{ Id = "crsp_87146"\n Eom = 29/02/2000 00:00:00\n Signal = 0.0095226144 }] } Portfolio 1 Eom 2000-02-01 00:00:00Z Stocks index value 0 { Id = "crsp_87146"\n Eom = 29/02/2000 00:00:00\n Signal = 0.0095226144 } Id crsp_87146 Eom 2000-02-29 00:00:00Z Signal 0.0095226144 2 { Portfolio = 1\n Eom = 01/03/2000 00:00:00\n Stocks = [{ Id = "crsp_87146"\n Eom = 31/03/2000 00:00:00\n Signal = 0.0095226144 }] } Portfolio 1 Eom 2000-03-01 00:00:00Z Stocks index value 0 { Id = "crsp_87146"\n Eom = 31/03/2000 00:00:00\n Signal = 0.0095226144 } Id crsp_87146 Eom 2000-03-31 00:00:00Z Signal 0.0095226144 3 { Portfolio = 1\n Eom = 01/04/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 30/04/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-04-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 30/04/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-04-30 00:00:00Z Signal 0.0114049256 4 { Portfolio = 1\n Eom = 01/05/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 31/05/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-05-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 31/05/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-05-31 00:00:00Z Signal 0.0114049256 5 { Portfolio = 1\n Eom = 01/06/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 30/06/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-06-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 30/06/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-06-30 00:00:00Z Signal 0.0114049256 6 { Portfolio = 1\n Eom = 01/07/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 31/07/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-07-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 31/07/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-07-31 00:00:00Z Signal 0.0114049256 7 { Portfolio = 1\n Eom = 01/08/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 31/08/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-08-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 31/08/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-08-31 00:00:00Z Signal 0.0114049256 8 { Portfolio = 1\n Eom = 01/09/2000 00:00:00\n Stocks = [{ Id = "crsp_82654"\n Eom = 30/09/2000 00:00:00\n Signal = 0.0114049256 }] } Portfolio 1 Eom 2000-09-01 00:00:00Z Stocks index value 0 { Id = "crsp_82654"\n Eom = 30/09/2000 00:00:00\n Signal = 0.0114049256 } Id crsp_82654 Eom 2000-09-30 00:00:00Z Signal 0.0114049256 9 { Portfolio = 1\n Eom = 01/10/2000 00:00:00\n Stocks = [{ Id = "crsp_24109"\n Eom = 31/10/2000 00:00:00\n Signal = 0.0088561242 }] } Portfolio 1 Eom 2000-10-01 00:00:00Z Stocks index value 0 { Id = "crsp_24109"\n Eom = 31/10/2000 00:00:00\n Signal = 0.0088561242 } Id crsp_24109 Eom 2000-10-31 00:00:00Z Signal 0.0088561242 10 { Portfolio = 1\n Eom = 01/11/2000 00:00:00\n Stocks = [{ Id = "crsp_24109"\n Eom = 30/11/2000 00:00:00\n Signal = 0.0088561242 }] } Portfolio 1 Eom 2000-11-01 00:00:00Z Stocks index value 0 { Id = "crsp_24109"\n Eom = 30/11/2000 00:00:00\n Signal = 0.0088561242 } Id crsp_24109 Eom 2000-11-30 00:00:00Z Signal 0.0088561242 11 { Portfolio = 1\n Eom = 01/12/2000 00:00:00\n Stocks = [{ Id = "crsp_24109"\n Eom = 31/12/2000 00:00:00\n Signal = 0.0088561242 }] } Portfolio 1 Eom 2000-12-01 00:00:00Z Stocks index value 0 { Id = "crsp_24109"\n Eom = 31/12/2000 00:00:00\n Signal = 0.0088561242 } Id crsp_24109 Eom 2000-12-31 00:00:00Z Signal 0.0088561242 12 { Portfolio = 1\n Eom = 01/01/2001 00:00:00\n Stocks = [{ Id = "crsp_88

> **Task:** Create a list named `bottomQuintileReturn` that contains the return of the bottom quintile portfolio each month. The portfolio return for a given month should be calculated using equal weights on every stock in the portfolio that month. The result should be given as a list of `SortedPortfolioReturn` records. **Additionally,** the month of the return should be lagged one month relative to the portfolio formation month. That means that if you formed a portfolio based on a signal known as of the end of February 2022 (Eom = DateTime(2022,02,28)), the portfolio return during the first month that you hold it will be calculated using stock returns during March 2022 (MonthOfReturn = DateTime(2022,03,31)).
> 

Quick example getting end of month addition:



In [ ]:

let endOfFebruary = DateTime(2022,02,28)

let addOneEom (eom: DateTime) =
    DateTime(eom.Year, eom.Month, 1).AddMonths(2).AddDays(-1.0)

addOneEom endOfFebruary

2022-03-31 00:00:00Z

That will give you the end of March. So in summary, if the signal that you use to form portfolios comes from February 2022 (signal EOM = DateTime(2022,2,28)), make sure that you get returns from March 2022 (return EOM = DateTime(2022,3,31)).

In [ ]:
// Defining position weights - equal weighting
let equalWeights x =
    let numStocks = List.length x.Stocks
    let weight = 1.0 / float numStocks
    List.map (fun _ -> weight) x.Stocks

// Fetching stock returns for a specific day
let fetchStockReturn (security, formationMonth) =
    let fallbackReturn = 0.0
    match Map.tryFind (security, formationMonth) byStockMonthIdAndReturnMap with
    | None -> fallbackReturn
    | Some x ->
        match x.Ret with
        | None -> fallbackReturn
        | Some r -> r

// Calculate portfolio return
type SortedPortfolioReturn =
    {
        Portfolio: int
        ReturnMonth: DateTime
        AvgReturn: float
    }

let bottomQuintileReturn: list<SortedPortfolioReturn> =
    quintiles
    |> List.filter (fun x -> x.Portfolio = 1)
    |> List.map (fun position ->
        let returns = position.Stocks |> List.map (fun stock -> fetchStockReturn (stock.Id, position.Eom))
        let weights = equalWeights position
        let weightedReturns = List.map2 (*) weights returns
        let avgReturn = List.average weightedReturns
        {
            Portfolio = position.Portfolio
            ReturnMonth = position.Eom.AddMonths(1) // Shift one month forward
            AvgReturn = avgReturn
        })

bottomQuintileReturn


index value 0 { Portfolio = 1\n ReturnMonth = 01/01/2009 00:00:00\n AvgReturn = -0.00042124 } Portfolio 1 ReturnMonth 2009-01-01 00:00:00Z AvgReturn -0.0004212399999999991 1 { Portfolio = 1\n ReturnMonth = 01/02/2000 00:00:00\n AvgReturn = -0.0080082 } Portfolio 1 ReturnMonth 2000-02-01 00:00:00Z AvgReturn -0.008008200000000002 2 { Portfolio = 1\n ReturnMonth = 01/01/2011 00:00:00\n AvgReturn = 0.01464492 } Portfolio 1 ReturnMonth 2011-01-01 00:00:00Z AvgReturn 0.01464492 3 { Portfolio = 1\n ReturnMonth = 01/11/2007 00:00:00\n AvgReturn = -0.00173504 } Portfolio 1 ReturnMonth 2007-11-01 00:00:00Z AvgReturn -0.0017350400000000002 4 { Portfolio = 1\n ReturnMonth = 01/07/2002 00:00:00\n AvgReturn = -0.00979108 } Portfolio 1 ReturnMonth 2002-07-01 00:00:00Z AvgReturn -0.00979108 5 { Portfolio = 1\n ReturnMonth = 01/08/2008 00:00:00\n AvgReturn = 0.03025304 } Portfolio 1 ReturnMonth 2008-08-01 00:00:00Z AvgReturn 0.03025304 6 { Portfolio = 1\n ReturnMonth = 01/06/2003 00:00:00\n AvgReturn = 0.01640752 } Portfolio 1 ReturnMonth 2003-06-01 00:00:00Z AvgReturn 0.016407520000000002 7 { Portfolio = 1\n ReturnMonth = 01/07/2003 00:00:00\n AvgReturn = 0.00077332 } Portfolio 1 ReturnMonth 2003-07-01 00:00:00Z AvgReturn 0.0007733200000000003 8 { Portfolio = 1\n ReturnMonth = 01/08/2003 00:00:00\n AvgReturn = -0.00663084 } Portfolio 1 ReturnMonth 2003-08-01 00:00:00Z AvgReturn -0.006630840000000001 9 { Portfolio = 1\n ReturnMonth = 01/09/2003 00:00:00\n AvgReturn = 0.003915 } Portfolio 1 ReturnMonth 2003-09-01 00:00:00Z AvgReturn 0.003915000000000001 10 { Portfolio = 1\n ReturnMonth = 01/10/2003 00:00:00\n AvgReturn = 0.00205544 } Portfolio 1 ReturnMonth 2003-10-01 00:00:00Z AvgReturn 0.00205544 11 { Portfolio = 1\n ReturnMonth = 01/11/2003 00:00:00\n AvgReturn = 0.01298392 } Portfolio 1 ReturnMonth 2003-11-01 00:00:00Z AvgReturn 0.012983920000000001 12 { Portfolio = 1\n ReturnMonth = 01/12/2003 00:00:00\n AvgReturn = -0.000641 } Portfolio 1 ReturnMonth 2003-12-01 00:00:00Z AvgReturn -0.000641 13 { Portfolio = 1\n ReturnMonth = 01/01/2004 00:00:00\n AvgReturn = 0.00669088 } Portfolio 1 ReturnMonth 2004-01-01 00:00:00Z AvgReturn 0.006690880000000001 14 { Portfolio = 1\n ReturnMonth = 01/02/2004 00:00:00\n AvgReturn = 0.00220876 } Portfolio 1 ReturnMonth 2004-02-01 00:00:00Z AvgReturn 0.00220876 15 { Portfolio = 1\n ReturnMonth = 01/03/2004 00:00:00\n AvgReturn = 0.0070516 } Portfolio 1 ReturnMonth 2004-03-01 00:00:00Z AvgReturn 0.0070516 16 { Portfolio = 1\n ReturnMonth = 01/04/2004 00:00:00\n AvgReturn = -0.00239832 } Portfolio 1 ReturnMonth 2004-04-01 00:00:00Z AvgReturn -0.0023983200000000002 17 { Portfolio = 1\n ReturnMonth = 01/05/2004 00:00:00\n AvgReturn = -0.00472492 } Portfolio 1 ReturnMonth 2004-05-01 00:00:00Z AvgReturn -0.0047249200000000005 18 { Portfolio = 1\n ReturnMonth = 01/06/2004 00:00:00\n AvgReturn = -0.00275588 } Portfolio 1 ReturnMonth 2004-06-01 00:00:00Z AvgReturn -0.002755880000000001 19 { Portfolio = 1\n ReturnMonth = 01/07/2004 00:00:00\n AvgReturn = 0.00309548 } Portfolio 1 ReturnMonth 2004-07-01 00:00:00Z AvgReturn 0.0030954800000000003 ... (more)

> **Task:** Plot a line chart of the cumulative return of the bottom quintile portfolio during the sample. For reference you will find the [plotting returns](https://nhirschey.github.io/Teaching/Momentum-Class.html#Plotting-returns) section of the momentum class lecture useful. It provides an example of calculating a portfolio's cumulative returns using `List.scan`.
> 



In [ ]:
let bottomCumRet =
    bottomQuintileReturn
    |> List.scan (fun (cumulativeReturn, _) month ->
        let newCumulativeReturn = cumulativeReturn * (1.0 + month.AvgReturn)
        (newCumulativeReturn, month.ReturnMonth)) (1.0, DateTime.MinValue)
    |> List.tail // Remove the initial seed value

let returnMonths, cumReturns = List.unzip (List.map (fun (cumulativeReturn, month) -> month, cumulativeReturn - 1.0) bottomCumRet)

let bottomQuintileCumRetPlot =
    Chart.Line(returnMonths, cumReturns)
    |> Chart.withTitle "Cumulative Return of Bottom Quintile Portfolio"
    |> Chart.withXAxisStyle("Month")
    |> Chart.withYAxisStyle("Cumulative Return")

bottomQuintileCumRetPlot


<!-- Plotly chart will be drawn inside this DIV -->